In [1]:
import pandas as pd
unagg = pd.read_csv("/mydata/q5.csv")
group_keys = unagg["n_name"].unique()
group_keys, unagg

(array(['CHINA                    ', 'INDIA                    ',
        'INDONESIA                ', 'JAPAN                    ',
        'VIETNAM                  '], dtype=object),
                             n_name     revenue   l_pageid  o_pageid
 0        CHINA                      13873.3668         12         3
 1        CHINA                      72690.6180        121        27
 2        CHINA                      45979.7580        151        35
 3        CHINA                      89343.6110        224        52
 4        CHINA                      13281.1560        234        54
 ...                            ...         ...        ...       ...
 6718281  VIETNAM                    50839.8800  115517549  26843200
 6718282  VIETNAM                    16274.1600  115517571  26843205
 6718283  VIETNAM                    11401.4673  115517641  26843221
 6718284  VIETNAM                    42675.8150  115517652  26843224
 6718285  VIETNAM                    65071.7466  1155177

In [2]:
unagg_china = unagg[unagg['n_name'] == 'CHINA                    ']
unagg_china

,n_name,revenue,l_pageid,o_pageid
0,CHINA,13873.3668,12,3
1,CHINA,72690.6180,121,27
2,CHINA,45979.7580,151,35
3,CHINA,89343.6110,224,52
4,CHINA,13281.1560,234,54
...,...,...,...,...
1344356,CHINA,64183.8960,115517500,26843189
1344357,CHINA,15169.7052,115517515,26843192
1344358,CHINA,86915.5000,115517530,26843196
1344359,CHINA,16788.4780,115517698,26843234


In [3]:
unagg_china.groupby("l_pageid").sum()["revenue"]

l_pageid
12           13873.3668
121          72690.6180
151          45979.7580
224          89343.6110
234          13281.1560
                ...    
115517500    64183.8960
115517515    15169.7052
115517530    86915.5000
115517698    16788.4780
115517704    50992.6684
Name: revenue, Length: 1343735, dtype: float64

In [6]:
import numpy as np
y_empty = (unagg_china["revenue"].sum() ** 2).item()
y_l = np.sum(unagg_china.groupby("l_pageid").sum()["revenue"] ** 2).item()
y_o = np.sum(unagg_china.groupby("o_pageid").sum()["revenue"] ** 2).item()
y_lo = np.sum(unagg_china["revenue"] ** 2).item()
sum_china = unagg_china["revenue"].sum()
y_empty, y_l, y_o, y_lo, sum_china

(2.794452412022734e+21,
 2949293638606829.0,
 3042138631903567.0,
 2947340945558816.0,
 52862580451.797226)

In [8]:
from scipy.stats import norm
z_val = norm.ppf(0.999)
max_var = ((sum_china * 0.05) / z_val) **2
max_var

7.31567611774731e+17

In [11]:
def get_var(theta1, theta2):
    a = theta1 * theta2
    b_empty = (theta1 ** 2) * (theta2 ** 2)
    b_l = theta1 * (theta2 **2)
    b_o = (theta1 ** 2) * theta2
    b_lo = theta1 * theta2
    c_empty = b_empty
    c_l = -b_empty + b_l
    c_o = -b_empty + b_o
    c_lo = b_empty - b_l - b_o + b_lo
    return c_empty / (a**2) * y_empty + c_l / (a**2) * y_l + c_o / (a**2) * y_o + c_lo / (a**2) * y_lo - y_empty

search_space = [0.0001 * i for i in range(1, 10)] + [0.001 * i for i in range(1, 10)] + [0.01 * i for i in range(1, 10)] + [0.1 * i for i in range(1, 6)]

for theta1 in search_space:
    var = get_var(theta1, 1)
    if var < max_var:
        print(f"single table sampling lineitem sample rate {theta1*100:.2f}%, variance {var:.2f}")
        break

for theta1 in search_space:
    for theta2 in search_space:
        var = get_var(theta1, theta2)
        if var < max_var:
            print(f"lineitem sample rate {theta1*100:.2f}%, order sample rate {theta2*100:.2f}%, variance {var:.2f}")


single table sampling lineitem sample rate 0.50%, variance 586909434083016704.00
lineitem sample rate 0.90%, order sample rate 50.00%, variance 652327124510179328.00
lineitem sample rate 1.00%, order sample rate 50.00%, variance 586808962464088064.00
lineitem sample rate 2.00%, order sample rate 30.00%, variance 488593026541682688.00
lineitem sample rate 2.00%, order sample rate 40.00%, variance 365708155738914816.00
lineitem sample rate 2.00%, order sample rate 50.00%, variance 291977233255890944.00
lineitem sample rate 3.00%, order sample rate 20.00%, variance 488718477802078208.00
lineitem sample rate 3.00%, order sample rate 30.00%, variance 324819318015524864.00
lineitem sample rate 3.00%, order sample rate 40.00%, variance 242869738123296768.00
lineitem sample rate 3.00%, order sample rate 50.00%, variance 193699990186491904.00
lineitem sample rate 4.00%, order sample rate 20.00%, variance 365896332628983808.00
lineitem sample rate 4.00%, order sample rate 30.00%, variance 242932

In [5]:
theta1 = 0.01
theta2 = 0.01
a = theta1 * theta2
b_empty = (theta1 ** 2) * (theta2 ** 2)
b_l = theta1 * (theta2 **2)
b_o = (theta1 ** 2) * theta2
b_lo = theta1 * theta2
c_empty = b_empty
c_l = -b_empty + b_l
c_o = -b_empty + b_o
c_lo = b_empty - b_l - b_o + b_lo
a, (b_empty, b_l, b_o, b_lo), (c_empty, c_l, c_o, c_lo)

(0.0001,
 (1e-08, 1.0000000000000002e-06, 1.0000000000000002e-06, 0.0001),
 (1e-08, 9.900000000000003e-07, 9.900000000000003e-07, 9.801e-05))

In [15]:
import math
print(c_empty / (a**2), y_empty)
print(c_l / (a**2), y_l)
print(c_o / (a**2), y_o)
print(c_lo / (a**2), y_lo)
sum_sigma2 = c_empty / (a**2) * y_empty + c_l / (a**2) * y_l + c_o / (a**2) * y_o + c_lo / (a**2) * y_lo - y_empty
sum_sigma2

1.0 2.794452412022734e+21
99.00000000000003 2949293638606829.0
99.00000000000003 3042138631903567.0
9801.0 2947340945558816.0


2.9480040402202395e+19

In [3]:
import glob
import numpy as np
result_files = glob.glob("results/*")
results = []
for result_file in result_files:
    with open(result_file) as f:
        result = f.readlines()
        if len(result) > 0:
            result = result[2]
            result = float(result.split("|")[-1].strip())
            results.append(result)
np.var(results, ddof=1), len(results)

(np.float64(4.917963431594842e+19), 42)

In [59]:
import pandas as pd
unagg = pd.read_csv("pilot_result_1.csv")
group_keys = unagg["n_name"].unique()
group_keys, unagg

(array(['CHINA                    ', 'INDIA                    ',
        'INDONESIA                ', 'JAPAN                    ',
        'VIETNAM                  '], dtype=object),
                         n_name   l_pageid  o_pageid     revenue
 0    CHINA                         123374     28669  54782.7660
 1    CHINA                         277119     64407   3023.8284
 2    CHINA                        1210536    281342  78016.2075
 3    CHINA                        2859326    664442   8001.0168
 4    CHINA                        2877724    668718  63842.8120
 ..                         ...        ...       ...         ...
 658  VIETNAM                    109049523  25340261  26666.5122
 659  VIETNAM                    109951720  25549904  28104.7410
 660  VIETNAM                    110263802  25622395  52133.3505
 661  VIETNAM                    115111551  26748860  48239.0349
 662  VIETNAM                    115179550  26764661  49713.5826
 
 [663 rows x 4 columns])

In [41]:
unagg_china = unagg[unagg['n_name'] == 'CHINA                    ']
unagg_china

,n_name,l_pageid,o_pageid,revenue
0,CHINA,123374,28669,54782.7660
1,CHINA,277119,64407,3023.8284
2,CHINA,1210536,281342,78016.2075
3,CHINA,2859326,664442,8001.0168
4,CHINA,2877724,668718,63842.8120
...,...,...,...,...
143,CHINA,113429303,26357987,51937.4124
144,CHINA,113512620,26377352,52564.1589
145,CHINA,114151748,26525861,41863.5486
146,CHINA,115001672,26723345,2857.3290


In [42]:
import numpy as np
y_empty = (unagg_china["revenue"].sum() ** 2).item()
y_l = np.sum(unagg_china.groupby("l_pageid").sum()["revenue"] ** 2).item()
y_o = np.sum(unagg_china.groupby("o_pageid").sum()["revenue"] ** 2).item()
y_lo = np.sum(unagg_china["revenue"] ** 2).item()
y_empty, y_l, y_o, y_lo

(27785811921078.13, 266162640275.9621, 266162640275.9621, 266162640275.9621)

In [43]:
theta1 = 0.01
theta2 = 0.01
a = theta1 * theta2
b_empty = (theta1 ** 2) * (theta2 ** 2)
b_l = theta1 * (theta2 **2)
b_o = (theta1 ** 2) * theta2
b_lo = theta1 * theta2
c_empty = b_empty
c_l = -b_empty + b_l
c_o = -b_empty + b_o
c_lo = b_empty - b_l - b_o + b_lo
a, (b_empty, b_l, b_o, b_lo), (c_empty, c_l, c_o, c_lo)

(0.0001,
 (1e-08, 1.0000000000000002e-06, 1.0000000000000002e-06, 0.0001),
 (1e-08, 9.900000000000003e-07, 9.900000000000003e-07, 9.801e-05))

In [47]:
import math
sum_sigma2 = c_empty / (a**2) * y_empty + c_l / (a**2) * y_l + c_o / (a**2) * y_o + c_lo / (a**2) * y_lo - y_empty
sum_sigma2

2661360240119345.0